In [4]:
import os
from nltk.tag import UnigramTagger
from nltk.corpus import treebank
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
import pandas as pd

In [9]:
path=r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\twitter_data_161122.csv'

In [10]:
df_init = pd.read_csv(path,sep=',',encoding='utf-8')
df_init.head()

,Datetime,Tweet Id,Text,Username
0,2022-11-16 23:59:43+00:00,1593031089961869315,@Guy_Smiley_73 @houndfucker @Indiekindoflife @...,julinnette1
1,2022-11-16 23:59:33+00:00,1593031045934301185,"GL:Well, don’t commit suicide.「自殺しないことです」(S1E1)",Sherlock_meigen
2,2022-11-16 23:59:20+00:00,1593030994600538112,@RealGomezMedia @iblockliberals @WickedR3coil ...,SIRLIGHTANGEL
3,2022-11-16 23:59:06+00:00,1593030935087575041,@Brjl_ おはよ！,Suicide429
4,2022-11-16 23:59:04+00:00,1593030924585021440,whilst i do not encourage misogyny: misandry k...,darthpilled


In [11]:
df=df_init.copy()

# NLP

In [18]:
def Predire(text):
    '''
    if type=='adverbe':
        Ltag=['RB','RBR','RBS','RBW']
        senti='r'
    elif type=='verbe':
        Ltag=['VB','VBD','VBG','VBN','VBP','VBZ']
        senti='v'
    '''

    train_sents = treebank.tagged_sents()[:3000]
    tagger = UnigramTagger(train_sents)

    texte=word_tokenize(text)
    L=set()
    tag=tagger.tag(texte)

    for t in tag:
        L.add(t[0])
        
    pos_score=0
    neg_score=0

    print(L)
    for adv in L:
        adv_senti = list(swn.senti_synsets(adv))
        if len(adv_senti)!=0:
            pos_score+=adv_senti[0].pos_score()
            neg_score+=adv_senti[0].neg_score()


    res=''
    if pos_score > neg_score:
        res='pos'
    elif pos_score < neg_score:
        res='neg'

    return res,pos_score,neg_score

In [19]:
test=df.loc[0]['Text']
print(test)
Predire(test)

@Guy_Smiley_73 @houndfucker @Indiekindoflife @AlessandraVaron But they do! They’re the reason that the suicide rate in the lgbt community is higher and that’s just a fact
{'they', 'AlessandraVaron', 'a', 'community', 'Indiekindoflife', 's', 'just', '’', '@', 'higher', 're', 'They', 'that', 'is', 'But', 'suicide', '!', 'the', 'do', 'rate', 'Guy_Smiley_73', 'in', 'and', 'reason', 'lgbt', 'houndfucker', 'fact'}


('pos', 1.375, 0.25)

In [ ]:
for i in range(10):
    print('--------------')
    print(df.loc[i]['Text'])
    print(Predire(df.loc[i]['Text']))
    print('--------------')

In [35]:
pos=0
neg=0
neutre=0
for index,row in df.iterrows():
    if Predire(row['Text'])[0] == 'pos':
        pos+=1
    elif Predire(row['Text'])[0] == 'neg':
        neg+=1
    else:
        neutre+=1

print('total= ', len(df))
print(pos,neg,neutre)

total=  18992
7117 6991 4884


# Valder

Lien pour les 3 modèles: 

https://towardsdatascience.com/the-most-favorable-pre-trained-sentiment-classifiers-in-python-9107c06442c6

In [17]:
#install vaderSentimentlibrary
#!pip install vaderSentiment

#import the library
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [18]:
#calculate the negative, positive, neutral and compound scores, plus verbal evaluation
def sentiment_vader(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"
        
    elif sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"   

    else :
        overall_sentiment = "Neutral"
  
    return negative, neutral, positive, compound, overall_sentiment

In [26]:
print(df.loc[0,'Text'])
sentiment_vader(df.loc[0,'Text'])

@Guy_Smiley_73 @houndfucker @Indiekindoflife @AlessandraVaron But they do! They’re the reason that the suicide rate in the lgbt community is higher and that’s just a fact


(0.214, 0.786, 0.0, -0.8197, 'Negative')

In [37]:
pos=0
neg=0
neutre=0
for index,row in df.iterrows():
    if sentiment_vader(row['Text'])[4] == 'Positive':
        pos+=1
    elif sentiment_vader(row['Text'])[4] == 'Negative':
        neg+=1
    else:
        neutre+=1

print('total= ', len(df))
print(pos,neg,neutre)

total=  18992
2470 12563 3959


# TextBlob

In [41]:
# install textblob
#pip install textblob

# import it
from textblob import TextBlob

In [42]:
#call the classifier
def sentiment_texblob(row):
  
    classifier = TextBlob(row)
    polarity = classifier.sentiment.polarity #[-1,1] emotions
    subjectivity = classifier.sentiment.subjectivity # objectif [0,1] subjectif
    
    return polarity,subjectivity


In [44]:
print(df.loc[0,'Text'])
sentiment_texblob(df.loc[0,'Text'])

@Guy_Smiley_73 @houndfucker @Indiekindoflife @AlessandraVaron But they do! They’re the reason that the suicide rate in the lgbt community is higher and that’s just a fact


(0.25, 0.5)

# Happy Transormer

In [22]:
#!pip install happytransformer

from happytransformer import HappyTextClassification

AttributeError: module 'tensorflow.core.framework.types_pb2' has no attribute 'SerializedDType'

In [47]:
happy_tc = HappyTextClassification(model_type="DISTILBERT", model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

12/08/2022 18:28:27 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [55]:
print(df.loc[0,'Text'])
happy_tc.classify_text(df.loc[0,'Text'])

@Guy_Smiley_73 @houndfucker @Indiekindoflife @AlessandraVaron But they do! They’re the reason that the suicide rate in the lgbt community is higher and that’s just a fact


TextClassificationResult(label='NEGATIVE', score=0.9565466642379761)

In [57]:
pos=0
neg=0
neutre=0
for index,row in df.iterrows():   
    if happy_tc.classify_text(row['Text']).label == 'NEGATIVE':
        neg+=1
    elif happy_tc.classify_text(row['Text']).label == 'POSITIVE':
        pos+=1
    else:
        neutre+=1

print('total= ', len(df))
print(pos,neg,neutre)

total=  18992
2208 16784 0


# Comparaison des modèles

In [69]:
import time

In [68]:
df.head()
df2=df_init.copy()

In [71]:
t1=time.perf_counter()

df2['Label_NLP']=df2.apply(lambda row: Predire(row['Text'])[0], axis=1)
df2['Label_Valder']=df2.apply(lambda row: sentiment_vader(row['Text'])[4], axis=1)
df2['Polarity']=df2.apply(lambda row: sentiment_texblob(row['Text'])[0], axis=1)
df2['Subjectivity']=df2.apply(lambda row: sentiment_texblob(row['Text'])[1], axis=1)
df2['Label_happy_transformer']=df2.apply(lambda row: happy_tc.classify_text(row['Text']).label, axis=1)

t2=time.perf_counter()

print(t2-t1)

1220.3325100000002


20 min

In [72]:
df2.head()

,Datetime,Tweet Id,Text,Username,Label_NLP,Label_Valder,Polarity,Subjectivity,Label_happy_transformer
0,2022-11-16 23:59:43+00:00,1593031089961869315,@Guy_Smiley_73 @houndfucker @Indiekindoflife @...,julinnette1,pos,Negative,0.250,1.375,NEGATIVE
1,2022-11-16 23:59:33+00:00,1593031045934301185,"GL:Well, don’t commit suicide.「自殺しないことです」(S1E1)",Sherlock_meigen,pos,Positive,0.000,0.125,NEGATIVE
2,2022-11-16 23:59:20+00:00,1593030994600538112,@RealGomezMedia @iblockliberals @WickedR3coil ...,SIRLIGHTANGEL,neg,Positive,0.600,0.000,NEGATIVE
3,2022-11-16 23:59:06+00:00,1593030935087575041,@Brjl_ おはよ！,Suicide429,,Neutral,0.000,0.000,NEGATIVE
4,2022-11-16 23:59:04+00:00,1593030924585021440,whilst i do not encourage misogyny: misandry k...,darthpilled,pos,Negative,0.125,2.875,POSITIVE
